Ler arquivos wpp.txt

In [197]:
import re
from pathlib import Path

def filtrar_relatorio(arquivo_entrada, arquivo_saida):
    # Define padrões para capturar seções importantes
    padrao_relevante = re.compile(r"(RETORNO DAS ATIVIDADES|HORÁRIOS|CANCELAMENTO|DESVIO DE ATIVIDADE|PTS|BLOQUEIO|LIMPEZA|AGUARDANDO ORDEM DE SERVIÇO|PET|DATA DE EXECUÇÃO|STATUS DA EXECUÇÃO|TAG|ORDEM|DESCRIÇÃO|TIPO DE ANDAIME|QTD\. DE PESSOAS|COMPRIMENTO|LARGURA|ALTURA|HORÁRIO)")
    padrao_irrelevante = re.compile(r"^\d{2}/\d{2}/\d{4} \d{2}:\d{2} - .+:.*")

    # Contadores para diagnóstico
    linhas_processadas = 0
    linhas_copiadas = 0
    linhas_ignoradas = 0

    try:
        # Verifica se o arquivo de entrada existe
        if not arquivo_entrada.exists():
            print(f"Erro: Arquivo de entrada '{arquivo_entrada}' não encontrado.")
            return

        with arquivo_entrada.open("r", encoding="utf-8") as entrada, arquivo_saida.open("w", encoding="utf-8") as saida:
            copiar = False
            for linha in entrada:
                linhas_processadas += 1  # Incrementa o total de linhas processadas
                # Verifica se a linha é relevante
                if padrao_relevante.search(linha):
                    copiar = True
                    print(f"-> Linha relevante encontrada: {linha.strip()}")
                elif padrao_irrelevante.match(linha):
                    copiar = False
                    print(f"Ignorando linha irrelevante: {linha.strip()}")
                    linhas_ignoradas += 1
                    continue  # Passa para a próxima linha

                # Copia linhas relevantes
                if copiar:
                    saida.write(linha)
                    linhas_copiadas += 1
    except Exception as e:
        print(f"Erro durante o processamento: {e}")
        return

    # Diagnóstico final
    print("\nProcessamento concluído!")
    print(f"Linhas processadas: {linhas_processadas}")
    print(f"Linhas copiadas para o relatório: {linhas_copiadas}")
    print(f"Linhas ignoradas: {linhas_ignoradas}")

# Define o diretório atual e os caminhos relativos dos arquivos
diretorio_atual = Path.cwd()  # Obtém o diretório de trabalho atual
arquivo_entrada = diretorio_atual / "wpp.txt"
arquivo_saida = diretorio_atual / "relatorio_filtrado.txt"

# Executa o filtro
filtrar_relatorio(arquivo_entrada, arquivo_saida)


Ignorando linha irrelevante: 23/01/2025 01:47 - Paulo Super Esp: ---------------------------------------------------
-> Linha relevante encontrada: RETORNO DAS ATIVIDADES
-> Linha relevante encontrada: DATA DE EXECUÇÃO: 22/01
-> Linha relevante encontrada: STATUS DA EXECUÇÃO:Desmontagem
-> Linha relevante encontrada: TAG: 2PE03
-> Linha relevante encontrada: ORDEM:202500155907
-> Linha relevante encontrada: DESCRIÇÃO:
-> Linha relevante encontrada: TIPO DE ANDAIME:Passarela
-> Linha relevante encontrada: COMPRIMENTO: 3,20
-> Linha relevante encontrada: LARGURA:1,05
-> Linha relevante encontrada: ALTURA:4,00
-> Linha relevante encontrada: HORÁRIOS
-> Linha relevante encontrada: HORÁRIO DE DISPONIBILIDADE:17:00
-> Linha relevante encontrada: HORÁRIO DE INÍCIO DA ATIVIDADE:17:10
-> Linha relevante encontrada: HORÁRIO DE FINALIZAÇÃO DA ATIVIDADE: 19:00
-> Linha relevante encontrada: CANCELAMENTO
-> Linha relevante encontrada: RAZÃO DO CANCELAMENTO:
-> Linha relevante encontrada: HORÁRIO DO

TRANSFORMAR .TXT EM .CSV

In [198]:
import csv
import re
from pathlib import Path

def gerar_csv(arquivo_entrada, arquivo_saida_csv):
    # Define as colunas do CSV
    colunas = [
        "DATA DE EXECUÇÃO", "STATUS DA EXECUÇÃO", "STATUS DO PLANEJAMENTO", "TAG",
        "ORDEM", "DESCRIÇÃO", "QTD PESSOAS", "TIPO DE ANDAIME", "QUANTIDADE DE ANDAIMES",
        "COMPRIMENTO", "LARGURA", "ALTURA",
        "HORÁRIO DE DISPONIBILIDADE", "HORÁRIO DE INÍCIO DA ATIVIDADE", "HORÁRIO DE FINALIZAÇÃO DA ATIVIDADE",
        "PREPARAÇÃO DE MATERIAL", "RAZÃO DO CANCELAMENTO", "HORÁRIO DO CANCELAMENTO",
        "RAZÃO DO DESVIO", "HORÁRIO DO DESVIO",
        "HORÁRIO DE SOLICITAÇÃO DA PTS", "HORÁRIO DE LIBERAÇÃO DA PTS",
        "HORÁRIO DE SOLICITAÇÃO DO BLOQUEIO", "HORÁRIO DE LIBERAÇÃO DO BLOQUEIO",
        "HORÁRIO DE SOLICITAÇÃO DA LIMPEZA", "HORÁRIO DE LIBERAÇÃO DA LIMPEZA",
        "HORÁRIO DE SOLICITAÇÃO DA OM", "HORÁRIO DE RECEBIMENTO DA OM",
        "HORÁRIO DE SOLICITAÇÃO DA PET", "HORÁRIO DE LIBERAÇÃO DA PET"
    ]

    # Define os padrões para capturar as informações
    padroes = {coluna: re.compile(fr"{coluna}: ?(.*)", re.IGNORECASE) for coluna in colunas}

    # Processa o arquivo e cria o CSV
    try:
        # Verifica se o arquivo de entrada existe
        if not arquivo_entrada.exists():
            print(f"Erro: Arquivo de entrada '{arquivo_entrada}' não encontrado.")
            return

        with arquivo_entrada.open("r", encoding="utf-8") as entrada, arquivo_saida_csv.open("w", encoding="utf-8", newline="") as saida_csv:
            writer = csv.DictWriter(saida_csv, fieldnames=colunas)
            writer.writeheader()

            dados_atual = {coluna: "" for coluna in colunas}  # Inicializa os dados de um relatório
            for linha in entrada:
                for coluna, padrao in padroes.items():
                    match = padrao.search(linha)
                    if match:
                        dados_atual[coluna] = match.group(1).strip()

                # Detecta o fim de um relatório para salvar os dados no CSV
                if "HORÁRIO DE LIBERAÇÃO DA PET" in linha.upper():
                    writer.writerow(dados_atual)
                    dados_atual = {coluna: "" for coluna in colunas}  # Reseta os dados para o próximo relatório

        print(f"Arquivo CSV gerado com sucesso em: {arquivo_saida_csv}")

    except Exception as e:
        print(f"Erro durante o processamento: {e}")

# Define o diretório atual e os caminhos relativos dos arquivos
diretorio_atual = Path.cwd()  # Obtém o diretório de trabalho atual
arquivo_entrada = diretorio_atual / "relatorio_filtrado.txt"
arquivo_saida_csv = diretorio_atual / "relatorio_final.csv"

# Executa o processamento
gerar_csv(arquivo_entrada, arquivo_saida_csv)


Arquivo CSV gerado com sucesso em: c:\Users\Administrador\Desktop\filtro espiral em jupyter\relatorio_final.csv


TRATAR COLUNA DE STATUS NO ARQUIVO.CSV

In [256]:
import pandas as pd
from pathlib import Path
import logging

# Configuração de logging para rastreamento detalhado
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def validar_formato_variacoes(caminho_arquivo):
    """Valida se o arquivo possui o formato esperado."""
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            linhas = [line.strip() for line in f if line.strip()]
            if len(linhas) < 3:
                logging.warning(f"O arquivo {caminho_arquivo} possui menos de 3 linhas.")
                return False
            if not linhas[0].startswith('| STATUS DA EXECUÇÃO CORRETO'):
                logging.warning(f"A primeira linha do arquivo {caminho_arquivo} não corresponde ao cabeçalho esperado.")
                return False
            if not any(line.startswith('| STATUS DA EXECUÇÃO INCORRETOS') for line in linhas):
                logging.warning(f"O arquivo {caminho_arquivo} não contém o cabeçalho para variações incorretas.")
                return False
        return True
    except Exception as e:
        logging.error(f"Erro ao validar o formato do arquivo {caminho_arquivo}: {e}")
        return False

def carregar_variacoes(caminho_arquivo):
    """Carrega as variações de erro de um arquivo de forma segura."""
    if not validar_formato_variacoes(caminho_arquivo):
        logging.warning(f"Formato inválido no arquivo de variações: {caminho_arquivo}")
        return []

    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            linhas = [line.strip().lower() for line in f if line.strip()]
            # Localiza a seção 'STATUS DA EXECUÇÃO INCORRETOS'
            for i, line in enumerate(linhas):
                if "| status da execução incorretos" in line:
                    logging.info(f"Cabeçalho 'STATUS DA EXECUÇÃO INCORRETOS' encontrado na linha {i} do arquivo {caminho_arquivo}.")
                    variacoes = linhas[i + 2:]  # Pula o cabeçalho e separador
                    # Remove o caractere '|' e espaços desnecessários
                    variacoes = [line.replace('|', '').strip() for line in variacoes if line.startswith('|')]
                    if variacoes:
                        logging.info(f"Variações carregadas de {caminho_arquivo}: {variacoes[:5]} (mostrando até 5 itens)")
                    else:
                        logging.warning(f"Nenhuma variação encontrada após o cabeçalho no arquivo {caminho_arquivo}.")
                    return variacoes
            logging.warning(f"Seção 'STATUS DA EXECUÇÃO INCORRETOS' não encontrada no arquivo {caminho_arquivo}.")
            return []
    except Exception as e:
        logging.error(f"Erro ao carregar as variações do arquivo {caminho_arquivo}: {e}")
        return []


def tratar_status(status, variacoes_status):
    """Trata os valores de 'STATUS DA EXECUÇÃO' para padronizar conforme variações, incluindo combinações."""
    status_original = status  # Preserve o valor original para depuração
    status = status.lower() if isinstance(status, str) else ""

    # Priorizar a categoria 'MONTAGEM E DESMONTAGEM'
    if 'MONTAGEM E DESMONTAGEM' in variacoes_status:
        variacoes_combinadas = variacoes_status['MONTAGEM E DESMONTAGEM']
        if any(variacao in status for variacao in variacoes_combinadas):
            logging.info(f"Status '{status_original}' padronizado para 'MONTAGEM E DESMONTAGEM'.")
            return 'MONTAGEM E DESMONTAGEM'

    # Verifica categorias individuais
    for status_corrente, variacoes in variacoes_status.items():
        if status_corrente != 'MONTAGEM E DESMONTAGEM':  # Ignora combinações nesta etapa
            if any(variacao in status for variacao in variacoes):
                logging.info(f"Status '{status_original}' padronizado para '{status_corrente}'.")
                return status_corrente

    # Adiciona nova variação se nenhuma correspondência for encontrada
    logging.warning(f"Status '{status_original}' não foi padronizado. Adicionando à lista de variações.")
    adicionar_nova_variacao(status_original)

    # Recarregar variações e tentar novamente
    for categoria, caminho in caminhos_variacoes.items():
        variacoes_status[categoria] = carregar_variacoes(caminho)

    # Repetir a lógica após recarregar
    return tratar_status(status_original, variacoes_status)

def adicionar_nova_variacao(status):
    """Adiciona um novo status não tratado ao arquivo correspondente."""
    status_lower = status.lower()

    # Verifica a qual categoria o status pertence
    for categoria, caminho in caminhos_variacoes.items():
        # Categoria é associada se o nome dela estiver no status
        if categoria.lower() in status_lower:
            try:
                with open(caminho, 'a', encoding='utf-8') as f:
                    f.write(f"| {status.strip()} |\n")
                logging.info(f"Nova variação '{status}' adicionada ao arquivo da categoria '{categoria}': {caminho}")
                return
            except Exception as e:
                logging.error(f"Erro ao adicionar a variação '{status}' ao arquivo {caminho}: {e}")
                return

    # Caso nenhuma correspondência seja encontrada, gerar um log de aviso
    logging.warning(f"Não foi possível associar o status '{status}' a nenhuma categoria existente.")

def adicionar_variacao(caminho_arquivo, nova_variacao):
    """Adiciona uma nova variação ao arquivo de variações."""
    try:
        with open(caminho_arquivo, 'a', encoding='utf-8') as f:
            f.write(f"{nova_variacao}\n")
        logging.info(f"Nova variação '{nova_variacao}' adicionada ao arquivo {caminho_arquivo}.")
    except Exception as e:
        logging.error(f"Erro ao adicionar variação ao arquivo {caminho_arquivo}: {e}")

# Diretório base dinâmico
base_dir = Path().resolve()

# Configuração dos caminhos
caminhos_variacoes = {
    'MONTAGEM': base_dir / "arquivos-gerados" / "arquivos-txt" / "variacoes-erros-montagem.txt",
    'DESMONTAGEM': base_dir / "arquivos-gerados" / "arquivos-txt" / "variacoes-erros-desmontagem.txt",
    'MONTAGEM E DESMONTAGEM': base_dir / "arquivos-gerados" / "arquivos-txt" / "variacoes-erros-montagem-e-desmontagem.txt",
    'ADEQUAÇÃO': base_dir / "arquivos-gerados" / "arquivos-txt" / "variacoes-erros-adequacoes.txt",
    'CANCELADO': base_dir / "arquivos-gerados" / "arquivos-txt" / "variacoes-erros-cancelado.txt",
    'EM ANDAMENTO': base_dir / "arquivos-gerados" / "arquivos-txt" / "variacoes-erros-em andamento.txt"
}

# Carregar as variações
variacoes_status = {
    status: carregar_variacoes(caminho) for status, caminho in caminhos_variacoes.items()
}

# Caminhos dos arquivos de entrada e saída
caminho_arquivo = base_dir / "relatorio_final.csv"
saida_dir = base_dir / "arquivos-gerados" / "arquivos-csv"
caminho_saida = saida_dir / "relatorio_tratado.csv"

# Garantir que o diretório de saída exista
saida_dir.mkdir(parents=True, exist_ok=True)

# Processamento do arquivo CSV
if not caminho_arquivo.exists():
    logging.error(f"Arquivo de entrada '{caminho_arquivo}' não encontrado.")
else:
    try:
        # Carregar o arquivo CSV
        df = pd.read_csv(caminho_arquivo)
        logging.info(f"Arquivo carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")

        # Padronizar os nomes das colunas
        df.columns = df.columns.str.strip()

        # Verificar e tratar a coluna 'STATUS DA EXECUÇÃO'
        if 'STATUS DA EXECUÇÃO' in df.columns:
            df['STATUS DA EXECUÇÃO'] = df['STATUS DA EXECUÇÃO'].apply(
                lambda x: tratar_status(x, variacoes_status)
            )
            logging.info("Correção da coluna 'STATUS DA EXECUÇÃO' realizada com sucesso.")

            # Salvar o arquivo tratado
            df.to_csv(caminho_saida, index=False)
            logging.info(f"Arquivo tratado salvo em: {caminho_saida}")
        else:
            logging.error("Coluna 'STATUS DA EXECUÇÃO' não encontrada no arquivo.")
    except Exception as e:
        logging.error(f"Erro durante o processamento do arquivo: {e}")


2025-01-25 13:23:42,935 - INFO - Cabeçalho 'STATUS DA EXECUÇÃO INCORRETOS' encontrado na linha 3 do arquivo C:\Users\Administrador\Desktop\filtro espiral em jupyter\arquivos-gerados\arquivos-txt\variacoes-erros-montagem.txt.
2025-01-25 13:23:42,937 - INFO - Variações carregadas de C:\Users\Administrador\Desktop\filtro espiral em jupyter\arquivos-gerados\arquivos-txt\variacoes-erros-montagem.txt: ['montagem andaime', 'montagn di andaim', 'montagen de andaime', 'motagen', 'muntagin'] (mostrando até 5 itens)
2025-01-25 13:23:42,939 - INFO - Cabeçalho 'STATUS DA EXECUÇÃO INCORRETOS' encontrado na linha 3 do arquivo C:\Users\Administrador\Desktop\filtro espiral em jupyter\arquivos-gerados\arquivos-txt\variacoes-erros-desmontagem.txt.
2025-01-25 13:23:42,940 - INFO - Variações carregadas de C:\Users\Administrador\Desktop\filtro espiral em jupyter\arquivos-gerados\arquivos-txt\variacoes-erros-desmontagem.txt: ['desmontagem andaime', 'desmontagen', 'desmonte', 'desmontagem de andami', 'desmont

Tratar Horas do arquivo.csv

In [200]:
import pandas as pd
from pathlib import Path

# Caminho do arquivo gerado anteriormente
diretorio_atual = Path.cwd()
caminho_arquivo = diretorio_atual / "relatorio_tratado.csv"

# Lista de colunas com campos de horas a serem padronizados
colunas_horas = [
    "HORÁRIO DE DISPONIBILIDADE", "HORÁRIO DE INÍCIO DA ATIVIDADE",
    "HORÁRIO DE FINALIZAÇÃO DA ATIVIDADE", "RAZÃO DO CANCELAMENTO",
    "HORÁRIO DO CANCELAMENTO", "HORÁRIO DO DESVIO",
    "HORÁRIO DE SOLICITAÇÃO DA PTS", "HORÁRIO DE LIBERAÇÃO DA PTS",
    "HORÁRIO DE SOLICITAÇÃO DO BLOQUEIO", "HORÁRIO DE LIBERAÇÃO DO BLOQUEIO",
    "HORÁRIO DE SOLICITAÇÃO DA LIMPEZA", "HORÁRIO DE LIBERAÇÃO DA LIMPEZA",
    "HORÁRIO DE SOLICITAÇÃO DA OM", "HORÁRIO DE RECEBIMENTO DA OM",
    "HORÁRIO DE SOLICITAÇÃO DA PET", "HORÁRIO DE LIBERAÇÃO DA PET"
]

# Função para padronizar os campos de hora
def padronizar_horas(valor):
    try:
        # Verifica se o valor é string e não está vazio
        if isinstance(valor, str) and valor.strip():
            # Se o formato for apenas "HH", completa para "HH:00:00"
            if len(valor) == 2 and valor.isdigit():
                return f"{valor}:00:00"
            # Se o formato for "HH:MM", completa para "HH:MM:00"
            elif len(valor) == 5 and ":" in valor:
                return f"{valor}:00"
            # Verifica se o valor já está no formato correto "HH:MM:SS"
            elif len(valor) == 8 and ":" in valor:
                return valor
        # Caso não seja válido, retorna o valor padrão "00:00:00"
        return "00:00:00"
    except Exception:
        return "00:00:00"

# Verifica se o arquivo de entrada existe
if not caminho_arquivo.exists():
    print(f"Erro: Arquivo '{caminho_arquivo}' não encontrado.")
else:
    # Carregar o arquivo .csv usando pandas
    df = pd.read_csv(caminho_arquivo)

    # Garantir que as colunas existem no DataFrame antes de aplicar a função
    for coluna in colunas_horas:
        if coluna in df.columns:
            df[coluna] = df[coluna].apply(padronizar_horas)

    # Caminho para o arquivo tratado
    caminho_saida = diretorio_atual / "relatorio_tratado_horas.csv"
    df.to_csv(caminho_saida, index=False)

    print(f"Arquivo tratado com horas padronizadas salvo em: {caminho_saida}")


Arquivo tratado com horas padronizadas salvo em: c:\Users\Administrador\Desktop\filtro espiral em jupyter\relatorio_tratado_horas.csv


Verificar se horas foram padronizadas corretamente

In [201]:
import pandas as pd
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Definir os caminhos relativos dos arquivos
caminho_arquivo = diretorio_atual / "relatorio_tratado.csv"
caminho_saida = diretorio_atual / "relatorio_tratado_horas.csv"

# Lista de colunas de horas para verificar
colunas_horas = [
    "HORÁRIO DE DISPONIBILIDADE", "HORÁRIO DE INÍCIO DA ATIVIDADE",
    "HORÁRIO DE FINALIZAÇÃO DA ATIVIDADE", "RAZÃO DO CANCELAMENTO",
    "HORÁRIO DO CANCELAMENTO", "HORÁRIO DO DESVIO",
    "HORÁRIO DE SOLICITAÇÃO DA PTS", "HORÁRIO DE LIBERAÇÃO DA PTS",
    "HORÁRIO DE SOLICITAÇÃO DO BLOQUEIO", "HORÁRIO DE LIBERAÇÃO DO BLOQUEIO",
    "HORÁRIO DE SOLICITAÇÃO DA LIMPEZA", "HORÁRIO DE LIBERAÇÃO DA LIMPEZA",
    "HORÁRIO DE SOLICITAÇÃO DA OM", "HORÁRIO DE RECEBIMENTO DA OM",
    "HORÁRIO DE SOLICITAÇÃO DA PET", "HORÁRIO DE LIBERAÇÃO DA PET"
]

# Função para padronizar horas
def padronizar_horas(valor):
    try:
        if isinstance(valor, str) and valor.strip():
            # Se o formato for "HH", completa para "HH:00:00"
            if len(valor) == 2 and valor.isdigit():
                return f"{valor}:00:00"
            # Se o formato for "HH:MM", completa para "HH:MM:00"
            elif len(valor) == 5 and ":" in valor:
                return f"{valor}:00"
            # Se já estiver no formato correto "HH:MM:SS", mantém
            elif len(valor) == 8 and ":" in valor:
                return valor
        # Caso inválido ou vazio, retorna "00:00:00"
        return "00:00:00"
    except Exception:
        return "00:00:00"

# Verificar se os arquivos existem
if not caminho_arquivo.exists():
    print(f"Erro: O arquivo de entrada '{caminho_arquivo}' não foi encontrado.")
elif not caminho_saida.exists():
    print(f"Erro: O arquivo tratado '{caminho_saida}' não foi encontrado.")
else:
    # Carregar os dois arquivos
    df_original = pd.read_csv(caminho_arquivo)
    df_tratado = pd.read_csv(caminho_saida)

    # Verificar alterações nas colunas de horas
    alteracoes = []
    for coluna in colunas_horas:
        if coluna in df_original.columns and coluna in df_tratado.columns:
            for index, (valor_original, valor_tratado) in enumerate(zip(df_original[coluna], df_tratado[coluna])):
                valor_padronizado = padronizar_horas(valor_original)
                if valor_padronizado != valor_tratado:
                    alteracoes.append({
                        "Linha": index + 1,
                        "Coluna": coluna,
                        "Valor Original": valor_original,
                        "Valor Tratado": valor_tratado,
                        "Esperado": valor_padronizado
                    })

    # Exibir ou salvar resultados
    if alteracoes:
        # Exibição limitada
        print("Foram encontradas inconsistências nas alterações realizadas:")
        for alteracao in alteracoes[:10]:  # Exibir apenas as primeiras 10 inconsistências
            print(f"Linha {alteracao['Linha']}, Coluna {alteracao['Coluna']}:")
            print(f" - Valor Original: {alteracao['Valor Original']}")
            print(f" - Valor Tratado: {alteracao['Valor Tratado']}")
            print(f" - Esperado: {alteracao['Esperado']}\n")
        if len(alteracoes) > 10:
            print(f"... e mais {len(alteracoes) - 10} inconsistências.")

        # Salvar relatório de inconsistências
        caminho_relatorio = diretorio_atual / "inconsistencias.csv"
        df_alteracoes = pd.DataFrame(alteracoes)
        df_alteracoes.to_csv(caminho_relatorio, index=False)
        print(f"Relatório de inconsistências salvo em: {caminho_relatorio}")
    else:
        print("Todas as alterações foram feitas corretamente.")


Todas as alterações foram feitas corretamente.


Tratar Dimensões

In [202]:
import pandas as pd
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Definir os caminhos relativos dos arquivos
caminho_arquivo = diretorio_atual / "relatorio_tratado_horas.csv"
caminho_saida = diretorio_atual / "relatorio_tratado_medidas.csv"

# Lista de colunas de medidas para padronizar
colunas_medidas = ["COMPRIMENTO", "LARGURA", "ALTURA"]

# Função para padronizar medidas
def padronizar_medidas(valor):
    try:
        # Verifica se o valor está vazio ou não numérico
        if pd.isna(valor) or str(valor).strip() == "":
            return "0,00"  # Preenche com 0,00
        # Converte para float e depois para o formato desejado
        valor_float = float(str(valor).replace(",", "."))
        return f"{valor_float:,.2f}".replace(".", ",")  # Padroniza para formato 0,00
    except Exception:
        return "0,00"  # Caso erro, retorna 0,00

# Verificar se o arquivo de entrada existe
if not caminho_arquivo.exists():
    print(f"Erro: O arquivo de entrada '{caminho_arquivo}' não foi encontrado.")
else:
    # Carregar o arquivo original
    df = pd.read_csv(caminho_arquivo)

    # Padronizar as colunas de medidas
    for coluna in colunas_medidas:
        if coluna in df.columns:
            df[coluna] = df[coluna].apply(padronizar_medidas)

    # Salvar o arquivo com as medidas padronizadas
    df.to_csv(caminho_saida, index=False)

    print(f"Arquivo com medidas padronizadas salvo em: {caminho_saida}")


Arquivo com medidas padronizadas salvo em: c:\Users\Administrador\Desktop\filtro espiral em jupyter\relatorio_tratado_medidas.csv


Verificar se dimensões foram tratadas corretamente

In [203]:
import pandas as pd
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Definir os caminhos relativos dos arquivos
caminho_etapa_4 = diretorio_atual / "relatorio_tratado.csv"
caminho_etapa_7 = diretorio_atual / "relatorio_tratado_medidas.csv"

# Lista de colunas de medidas para verificar
colunas_medidas = ["COMPRIMENTO", "LARGURA", "ALTURA"]

# Função para padronizar medidas
def padronizar_medidas(valor):
    try:
        # Verifica se o valor é um número válido
        if isinstance(valor, str) and valor.strip():
            # Se o valor for "NÃ" ou qualquer outra string não numérica, retorna 0,00
            if any(char.isalpha() for char in valor):
                return "0,00"
            # Substitui ponto por vírgula
            valor = valor.replace('.', ',')
            # Se o valor tiver uma vírgula, verificamos a quantidade de casas decimais
            if ',' in valor:
                # Caso o valor tenha apenas uma casa decimal, adiciona a segunda casa como "0"
                if valor.count(',') == 1 and len(valor.split(',')[1]) == 1:
                    return f"{valor}0"
                # Caso o valor já tenha duas casas decimais, retorna o valor como está
                elif valor.count(',') == 1 and len(valor.split(',')[1]) == 2:
                    return valor
                # Caso o valor tenha mais de duas casas decimais, arredonda para 2 casas
                else:
                    return f"{valor.split(',')[0]},{valor.split(',')[1][:2]}"
            else:
                return f"{valor},00"
        return "0,00"  # Se vazio ou inválido
    except Exception:
        return "0,00"  # Retorna "0,00" em caso de erro

# Verificar se o arquivo de entrada da etapa 4 existe
if not caminho_etapa_4.exists():
    print(f"Erro: O arquivo de entrada '{caminho_etapa_4}' não foi encontrado.")
else:
    # Carregar os dois arquivos
    df_etapa_4 = pd.read_csv(caminho_etapa_4)
    df_etapa_7 = pd.read_csv(caminho_etapa_7)

    # Verificar alterações nas colunas de medidas
    alteracoes = []
    for coluna in colunas_medidas:
        if coluna in df_etapa_4.columns and coluna in df_etapa_7.columns:
            for index, (valor_etapa_4, valor_etapa_7) in enumerate(zip(df_etapa_4[coluna], df_etapa_7[coluna])):
                valor_padronizado = padronizar_medidas(valor_etapa_4)
                # Comparar valor tratado com o valor padronizado
                if valor_padronizado != valor_etapa_7:
                    alteracoes.append({
                        "Linha": index + 1,
                        "Coluna": coluna,
                        "Valor Original": valor_etapa_4,
                        "Valor Tratado": valor_etapa_7,
                        "Esperado": valor_padronizado
                    })

    # Exibir resultados
    if alteracoes:
        print("Foram encontradas inconsistências nas alterações realizadas:")
        for alteracao in alteracoes:
            print(f"Linha {alteracao['Linha']}, Coluna {alteracao['Coluna']}:")
            print(f" - Valor Original: {alteracao['Valor Original']}")
            print(f" - Valor Tratado: {alteracao['Valor Tratado']}")
            print(f" - Esperado: {alteracao['Esperado']}\n")
    else:
        print("Todas as alterações foram feitas corretamente.")


Todas as alterações foram feitas corretamente.


Padronizar valores

In [204]:
import pandas as pd
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Definir os caminhos relativos dos arquivos
arquivo_entrada = diretorio_atual / "relatorio_tratado_medidas.csv"
arquivo_saida = diretorio_atual / "relatorio_tratado_qtd.csv"

# Lista de colunas para verificar
colunas_tratamento = ["QTD PESSOAS", "QUANTIDADE DE ANDAIMES"]

# Função para padronizar os valores
def padronizar_valores(valor):
    try:
        # Verifica se é NaN ou vazio
        if pd.isna(valor) or str(valor).strip() == "":
            return "0,00"
        # Verifica se é texto
        if isinstance(valor, str):
            valor = valor.strip().lower()
            if not valor.replace('.', '').replace(',', '').isdigit():
                return "0,00"
        # Conversão para float
        valor = float(valor)
        if valor in [1, 1.0, 1.00]:
            return "1,00"
        elif valor in [3, 3.0, 3.00]:
            return "3,00"
        elif valor in [10, 10.0, 10.00]:
            return "10,00"
        elif valor in [0, 0.0, 0.00]:
            return "0,00"
        # Retorna valores no formato esperado
        return f"{valor:.2f}".replace('.', ',')
    except Exception:
        return "0,00"  # Retorna "0,00" em caso de erro

# Verificar se o arquivo de entrada existe
if not arquivo_entrada.exists():
    print(f"Erro: O arquivo de entrada '{arquivo_entrada}' não foi encontrado.")
else:
    # Carregar o arquivo de entrada
    try:
        df_entrada = pd.read_csv(arquivo_entrada)
    except Exception as e:
        print(f"Erro ao carregar o arquivo de entrada: {e}")
        raise

    # Criar uma cópia do DataFrame para saída
    df_saida = df_entrada.copy()

    # Verificar e tratar as colunas
    for coluna in colunas_tratamento:
        if coluna in df_entrada.columns:
            # Padronizar valores na coluna
            df_saida[coluna] = df_entrada[coluna].apply(padronizar_valores)
        else:
            print(f"Coluna {coluna} não encontrada no arquivo de entrada.")

    # Salvar o arquivo tratado
    try:
        df_saida.to_csv(arquivo_saida, index=False)
        print(f"Arquivo tratado e salvo em: {arquivo_saida}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo de saída: {e}")


Arquivo tratado e salvo em: c:\Users\Administrador\Desktop\filtro espiral em jupyter\relatorio_tratado_qtd.csv


Verificar se tratamento foi executado corretamente

In [205]:
import pandas as pd
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Definir os caminhos relativos dos arquivos
arquivo_original = diretorio_atual / "relatorio_tratado.csv"
arquivo_tratado = diretorio_atual / "relatorio_tratado_qtd.csv"

# Lista de colunas para verificar
colunas_tratamento = ["QTD PESSOAS", "QUANTIDADE DE ANDAIMES"]

# Função para padronizar os valores
def padronizar_valores(valor):
    try:
        # Verifica se é NaN ou vazio
        if pd.isna(valor) or str(valor).strip() == "":
            return "0,00"
        # Verifica se é texto
        if isinstance(valor, str):
            valor = valor.strip().lower()
            if not valor.replace('.', '').replace(',', '').isdigit():
                return "0,00"
        # Conversão para float, se aplicável
        valor = float(valor)
        if valor == 1 or valor == 1.0 or valor == 1.00:
            return "1,00"
        elif valor == 3 or valor == 3.0 or valor == 3.00:
            return "3,00"
        elif valor == 10 or valor == 10.0 or valor == 10.00:
            return "10,00"
        elif valor == 0 or valor == 0.0 or valor == 0.00:
            return "0,00"
        # Retorna valores no formato esperado
        return f"{valor:.2f}".replace('.', ',')
    except Exception:
        return "0,00"  # Retorna "0,00" em caso de erro

# Verificar se os arquivos existem
if not arquivo_original.exists():
    print(f"Erro: O arquivo original '{arquivo_original}' não foi encontrado.")
else:
    if not arquivo_tratado.exists():
        print(f"Erro: O arquivo tratado '{arquivo_tratado}' não foi encontrado.")
    else:
        # Carregar os arquivos original e tratado
        try:
            df_original = pd.read_csv(arquivo_original)
            df_tratado = pd.read_csv(arquivo_tratado)
        except Exception as e:
            print(f"Erro ao carregar os arquivos: {e}")
            raise

        # Função para verificar se o valor foi alterado indevidamente
        def verificar_valor_tratado(valor_original, valor_tratado):
            # Padronizar o valor original para o formato esperado
            valor_esperado = padronizar_valores(valor_original)
            # Comparar os valores
            if str(valor_esperado).replace(',', '.').strip() != str(valor_tratado).replace(',', '.').strip():
                return False, valor_esperado
            return True, valor_esperado

        # Verificar inconsistências
        inconsistencias = []
        for coluna in colunas_tratamento:
            if coluna in df_original.columns and coluna in df_tratado.columns:
                for index, (valor_original, valor_tratado) in enumerate(zip(df_original[coluna], df_tratado[coluna])):
                    valido, valor_esperado = verificar_valor_tratado(valor_original, valor_tratado)
                    if not valido:
                        inconsistencias.append({
                            "Linha": index + 1,
                            "Coluna": coluna,
                            "Valor Original": valor_original,
                            "Valor Tratado": valor_tratado,
                            "Esperado": valor_esperado
                        })

        # Exibir resultados
        if inconsistencias:
            print("Foram encontradas inconsistências entre os arquivos:")
            for inc in inconsistencias:
                print(f"Linha {inc['Linha']}, Coluna {inc['Coluna']}:")
                print(f" - Valor Original: {inc['Valor Original']}")
                print(f" - Valor Tratado: {inc['Valor Tratado']}")
                print(f" - Esperado: {inc['Esperado']}\n")
        else:
            print("Nenhuma inconsistência foi encontrada entre os arquivos.")


Nenhuma inconsistência foi encontrada entre os arquivos.


Tratar Datas

In [206]:
import pandas as pd
from datetime import datetime
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Caminhos relativos dos arquivos de entrada e saída
arquivo_entrada = diretorio_atual / "relatorio_tratado_qtd.csv"
arquivo_saida = diretorio_atual / "relatorio_tratado_datas.csv"

# Função para padronizar as datas
def padronizar_data(data):
    try:
        # Remover espaços extras e barras duplicadas
        data = str(data).strip()
        # Remove qualquer espaço antes e depois das barras
        data = data.replace(" //", "/").replace("//", "/").replace("/ ", "/")
        data = data.replace(" ", "")  # Remover qualquer espaço extra em toda a data

        # Verificar se a data contém o ano (ex.: "03/01/2024" ou "03/01/25")
        if len(data.split("/")) == 3:  # Se tem 3 partes (dia/mês/ano)
            dia, mes, ano = data.split("/")
            # Corrigir anos com 2 dígitos, como "25" para "2025"
            if len(ano) == 2:
                ano = "20" + ano
            # Padronizar para o formato DD/MM/YYYY
            return datetime.strptime(f"{dia}/{mes}/{ano}", "%d/%m/%Y").strftime("%d/%m/%Y")

        # Corrigir datas com apenas dia e mês (como "06/12")
        if len(data.split("/")) == 2:
            data = data + "/" + str(datetime.now().year)  # Adicionar o ano atual
            return datetime.strptime(data, "%d/%m/%Y").strftime("%d/%m/%Y")

        return data
    except Exception as e:
        # Caso o valor não seja uma data válida, retorna o valor original
        return data

# Verificar se o arquivo de entrada existe
if not arquivo_entrada.exists():
    print(f"Erro: O arquivo de entrada '{arquivo_entrada}' não foi encontrado.")
else:
    # Carregar o arquivo de entrada
    try:
        df = pd.read_csv(arquivo_entrada)
    except Exception as e:
        print(f"Erro ao carregar o arquivo '{arquivo_entrada}': {e}")
        raise

    # Verificar a coluna de datas e padronizar
    if 'DATA DE EXECUÇÃO' in df.columns:
        for index, data in enumerate(df['DATA DE EXECUÇÃO']):
            data_padronizada = padronizar_data(data)
            # Se a data padronizada for diferente da original, vamos atualizar
            if data_padronizada != str(data).strip():
                df.at[index, 'DATA DE EXECUÇÃO'] = data_padronizada

    # Corrigir datas de 2024 que são inconsistentes
    for index, data in enumerate(df['DATA DE EXECUÇÃO']):
        try:
            data_formatada = datetime.strptime(data, "%d/%m/%Y")
            # Corrigir datas muito antigas, por exemplo, 03/01/2024 quando deveriam ser de 2025
            if data_formatada.year == 2024 and data_formatada < datetime(2025, 1, 1):
                df.at[index, 'DATA DE EXECUÇÃO'] = (data_formatada.replace(year=2025)).strftime("%d/%m/%Y")
        except Exception:
            continue  # Se a data não estiver no formato esperado, ignore a linha

    # Verificar se o arquivo de saída pode ser salvo
    if not arquivo_saida.parent.exists():
        print(f"Erro: O diretório de saída '{arquivo_saida.parent}' não existe.")
    else:
        # Salvar o arquivo corrigido
        try:
            df.to_csv(arquivo_saida, index=False)
            print(f"Datas corrigidas e arquivo atualizado com sucesso! Arquivo salvo em: {arquivo_saida}")
        except Exception as e:
            print(f"Erro ao salvar o arquivo '{arquivo_saida}': {e}")


Datas corrigidas e arquivo atualizado com sucesso! Arquivo salvo em: c:\Users\Administrador\Desktop\filtro espiral em jupyter\relatorio_tratado_datas.csv


Verificar datas tratadas

In [207]:
import pandas as pd
from datetime import datetime
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Caminhos relativos dos arquivos
caminho_arquivo_original = diretorio_atual / "relatorio_tratado_qtd.csv"
caminho_arquivo_tratado = diretorio_atual / "relatorio_tratado_datas.csv"

# Função para verificar a padronização da data com base nas regras
def verificar_data(data):
    try:
        data = str(data).strip()
        data = data.replace(" //", "/").replace("//", "/").replace("/ ", "/").replace(" ", "")

        if len(data.split("/")) == 3:
            dia, mes, ano = data.split("/")
            if len(ano) == 2:
                ano = "20" + ano
            # Garantir zeros à esquerda
            dia = dia.zfill(2)
            mes = mes.zfill(2)
            return f"{dia}/{mes}/{ano}"

        if len(data.split("/")) == 2:
            dia, mes = data.split("/")
            # Garantir zeros à esquerda e adicionar o ano atual
            dia = dia.zfill(2)
            mes = mes.zfill(2)
            return f"{dia}/{mes}/{datetime.now().year}"

        return data
    except Exception:
        return data

# Verificar se os arquivos de entrada existem
if not caminho_arquivo_original.exists():
    print(f"Erro: O arquivo original '{caminho_arquivo_original}' não foi encontrado.")
elif not caminho_arquivo_tratado.exists():
    print(f"Erro: O arquivo tratado '{caminho_arquivo_tratado}' não foi encontrado.")
else:
    # Carregar os arquivos
    try:
        df_original = pd.read_csv(caminho_arquivo_original)
        df_tratado = pd.read_csv(caminho_arquivo_tratado)
    except Exception as e:
        print(f"Erro ao carregar os arquivos: {e}")
        raise

    # Verificar inconsistências
    if 'DATA DE EXECUÇÃO' in df_original.columns and 'DATA DE EXECUÇÃO' in df_tratado.columns:
        inconsistencias = []

        for index, (data_original, data_tratada) in enumerate(zip(df_original['DATA DE EXECUÇÃO'], df_tratado['DATA DE EXECUÇÃO'])):
            # Determinar o valor esperado sem realizar modificações no dado original
            esperado = verificar_data(data_original)

            # Ajuste para anos inconsistentes
            try:
                data_formatada = datetime.strptime(esperado, "%d/%m/%Y")
                if data_formatada.year == 2024 and data_formatada < datetime(2025, 1, 1):
                    esperado = data_formatada.replace(year=2025).strftime("%d/%m/%Y")
            except Exception:
                pass

            # Verificar se o valor tratado corresponde ao esperado
            if esperado != data_tratada:
                inconsistencias.append({
                    "Linha": index + 1,
                    "Valor Original": data_original,
                    "Valor Tratado": data_tratada,
                    "Esperado": esperado
                })

        # Exibir inconsistências
        if inconsistencias:
            print("Foram encontradas inconsistências:")
            for inconsistencia in inconsistencias:
                print(f"Linha {inconsistencia['Linha']}:")
                print(f" - Valor Original: {inconsistencia['Valor Original']}")
                print(f" - Valor Tratado: {inconsistencia['Valor Tratado']}")
                print(f" - Esperado: {inconsistencia['Esperado']}\n")
        else:
            print("Todas as datas foram tratadas corretamente!")
    else:
        print("Coluna 'DATA DE EXECUÇÃO' não encontrada em um dos arquivos.")


Todas as datas foram tratadas corretamente!


Transformar para caixa alta

In [208]:
import pandas as pd
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Caminhos relativos para o arquivo de entrada e saída
arquivo_saida = diretorio_atual / "relatorio_tratado_datas.csv"
arquivo_caixa_alta = diretorio_atual / "relatorio_tratado_caixa_alta.csv"

# Verificar se o arquivo de entrada existe
if not arquivo_saida.exists():
    print(f"Erro: O arquivo original '{arquivo_saida}' não foi encontrado.")
else:
    try:
        # Carregar o arquivo original
        df = pd.read_csv(arquivo_saida)

        # Converter todas as colunas de texto para caixa alta
        df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

        # Salvar o novo arquivo com os dados em caixa alta
        df.to_csv(arquivo_caixa_alta, index=False)

        print(f"Arquivo gerado com sucesso em: {arquivo_caixa_alta}")
    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")


Arquivo gerado com sucesso em: c:\Users\Administrador\Desktop\filtro espiral em jupyter\relatorio_tratado_caixa_alta.csv


C:\Users\Administrador\AppData\Local\Temp\ipykernel_19536\3360258727.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x)


Verificar se foi devidamente tratado

In [209]:
import pandas as pd
from pathlib import Path

# Determinar o diretório de trabalho atual
diretorio_atual = Path.cwd()

# Caminhos relativos para o arquivo CSV de entrada e o arquivo Excel de saída
arquivo_csv = diretorio_atual / "relatorio_tratado_caixa_alta.csv"
arquivo_excel = diretorio_atual / "relatorio_tratado.xlsx"

# Verificar se o arquivo CSV existe
if not arquivo_csv.exists():
    print(f"Erro: O arquivo '{arquivo_csv}' não foi encontrado.")
else:
    try:
        # Ler o arquivo CSV e garantir que a coluna ORDEM seja tratada como string
        df = pd.read_csv(arquivo_csv, dtype={'ORDEM': str})

        # Salvar para Excel sem modificar os valores
        df.to_excel(arquivo_excel, index=False, engine='openpyxl')

        print(f"Arquivo Excel criado com sucesso em: {arquivo_excel}")
    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")


Arquivo Excel criado com sucesso em: c:\Users\Administrador\Desktop\filtro espiral em jupyter\relatorio_tratado.xlsx


Criar COLUNA de RETORNO DA EXECUÇÃO

In [210]:
import pandas as pd
from pathlib import Path

# Função para criar a coluna 'RETORNO DA EXECUÇÃO'
def criar_coluna_retorno(arquivo_excel):
    # Verificar se o arquivo Excel existe
    if not arquivo_excel.exists():
        print(f"Erro: O arquivo '{arquivo_excel}' não foi encontrado.")
        return

    # Carregar o arquivo Excel
    try:
        df = pd.read_excel(arquivo_excel)
        print("Arquivo carregado com sucesso.")
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return

    # Exibir as primeiras linhas para depuração
    print("Primeiras linhas do arquivo:")
    print(df.head())

    # Verificar se a coluna 'STATUS DA EXECUÇÃO' existe, removendo espaços extras
    if 'STATUS DA EXECUÇÃO' not in df.columns:
        print("A coluna 'STATUS DA EXECUÇÃO' não foi encontrada no arquivo. Verificando se há espaços extras...")
        df.columns = df.columns.str.strip()  # Remover espaços extras nos nomes das colunas
        if 'STATUS DA EXECUÇÃO' not in df.columns:
            print("A coluna 'STATUS DA EXECUÇÃO' ainda não foi encontrada após remoção de espaços.")
            return
        else:
            print("Espaços extras removidos. Coluna 'STATUS DA EXECUÇÃO' encontrada.")

    # Definir as palavras relacionadas a "CANCELADO"
    cancelado_palavras = ["CANCELADO", "CANCELADA", "CANCELL", "CANCEL", "CANCELO"]

    # Definir as palavras relacionadas a "EXECUTADO"
    executado_palavras = ["MONTADO", "DESMONTADO", "ADEQUAÇÃO", "EXECUTADO", "MONTAGEM", "DESMONTA"]

    # Criar a nova coluna 'RETORNO DA EXECUÇÃO' com valores default vazios
    df['RETORNO DA EXECUÇÃO'] = ""

    # Verificar o campo 'STATUS DA EXECUÇÃO' e atribuir valores à nova coluna
    for index, row in df.iterrows():
        status_execucao = str(row['STATUS DA EXECUÇÃO']).upper()  # Converter para maiúsculas para comparação

        # Verificar se o status está relacionado a "CANCELADO"
        if any(palavra in status_execucao for palavra in cancelado_palavras):
            df.at[index, 'RETORNO DA EXECUÇÃO'] = "CANCELADO"

        # Verificar se o status está relacionado a "EXECUTADO"
        elif any(palavra in status_execucao for palavra in executado_palavras):
            df.at[index, 'RETORNO DA EXECUÇÃO'] = "EXECUTADO"

    # Exibir as primeiras linhas após adicionar a coluna
    print("Primeiras linhas após adicionar a coluna 'RETORNO DA EXECUÇÃO':")
    print(df.head())

    # Salvar o arquivo modificado no mesmo local
    try:
        df.to_excel(arquivo_excel, index=False, engine='openpyxl')
        print(f"Coluna 'RETORNO DA EXECUÇÃO' adicionada com sucesso no arquivo: {arquivo_excel}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")


# Determinar o diretório de trabalho atual e o caminho relativo do arquivo
diretorio_atual = Path.cwd()
arquivo_excel = diretorio_atual / "relatorio_tratado.xlsx"

# Chamar a função para processar o arquivo
criar_coluna_retorno(arquivo_excel)


Arquivo carregado com sucesso.
Primeiras linhas do arquivo:
  DATA DE EXECUÇÃO              STATUS DA EXECUÇÃO STATUS DO PLANEJAMENTO  \
0       22/01/2025                     DESMONTAGEM             PROGRAMADO   
1       22/01/2025                     DESMONTAGEM             PROGRAMADO   
2       23/01/2025  MONTAGEM DE ANDAIME FINALIZADA             PROGRAMADO   
3       23/01/2025  MONTAGEM DE ANDAIME FINALIZADO              CORRETIVA   
4       23/01/2025                         MONTADO                    NaN   

            TAG         ORDEM  \
0         2PE03  202500155907   
1         2PE01  202500155806   
2  TC-6042-09TC  202500258690   
3       TC-6015  202500399129   
4        TC5007  202405254103   

                                           DESCRIÇÃO QTD PESSOAS  \
0                                                NaN        3,00   
1                                                NaN        3,00   
2  MONTAGEM DE ANDAIME PARA VOCALIZAÇÃO ACESSAR O...        4,00   
3  FOR

REORGANIZAR COLUNAS RETORNO DA EXECUÇÃO

In [211]:
import pandas as pd
from pathlib import Path

# Função para reorganizar as colunas conforme nova lógica
def reorganizar_colunas_nova_logica(arquivo_excel):
    # Verificar se o arquivo Excel existe
    if not arquivo_excel.exists():
        print(f"Erro: O arquivo '{arquivo_excel}' não foi encontrado.")
        return

    # Carregar o arquivo Excel
    try:
        df = pd.read_excel(arquivo_excel)
        print("Arquivo carregado com sucesso.")
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return

    # Exibir as primeiras linhas para depuração
    print("Primeiras linhas do arquivo:")
    print(df.head())

    # Verificar se existem pelo menos 31 colunas
    if len(df.columns) < 31:
        print("Erro: O arquivo precisa ter pelo menos 31 colunas.")
        return

    # Identificar a coluna de índice 31 (considerando a indexação 0)
    coluna_31 = df.columns[30]  # 31ª coluna (index 30)

    # Mover a coluna 31 para a 4ª posição (index 3)
    df.insert(3, coluna_31, df.pop(coluna_31))  # Move para a 4ª posição

    # Exibir as primeiras linhas após reorganizar
    print("Primeiras linhas após reorganizar as colunas:")
    print(df.head())

    # Salvar o arquivo modificado
    try:
        df.to_excel(arquivo_excel, index=False, engine='openpyxl')
        print(f"Colunas reorganizadas com sucesso em: {arquivo_excel}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")


# Determinar o diretório de trabalho atual e o caminho relativo do arquivo
diretorio_atual = Path.cwd()
arquivo_excel = diretorio_atual / "relatorio_tratado.xlsx"

# Chamada para reorganizar as colunas conforme a nova lógica
reorganizar_colunas_nova_logica(arquivo_excel)


Arquivo carregado com sucesso.
Primeiras linhas do arquivo:
  DATA DE EXECUÇÃO              STATUS DA EXECUÇÃO STATUS DO PLANEJAMENTO  \
0       22/01/2025                     DESMONTAGEM             PROGRAMADO   
1       22/01/2025                     DESMONTAGEM             PROGRAMADO   
2       23/01/2025  MONTAGEM DE ANDAIME FINALIZADA             PROGRAMADO   
3       23/01/2025  MONTAGEM DE ANDAIME FINALIZADO              CORRETIVA   
4       23/01/2025                         MONTADO                    NaN   

            TAG         ORDEM  \
0         2PE03  202500155907   
1         2PE01  202500155806   
2  TC-6042-09TC  202500258690   
3       TC-6015  202500399129   
4        TC5007  202405254103   

                                           DESCRIÇÃO QTD PESSOAS  \
0                                                NaN        3,00   
1                                                NaN        3,00   
2  MONTAGEM DE ANDAIME PARA VOCALIZAÇÃO ACESSAR O...        4,00   
3  FOR

Verificar a consistencial geral dos dados

In [212]:
import pandas as pd
from datetime import datetime
from pathlib import Path

# Função para carregar os dados de ambos os arquivos
def carregar_dados(arquivo_original, arquivo_tratado):
    try:
        # Verificar se os arquivos existem
        if not arquivo_original.exists() or not arquivo_tratado.exists():
            print("Erro: Um ou ambos os arquivos não foram encontrados.")
            return None, None

        # Carregar os dados
        df_original = pd.read_csv(arquivo_original)
        df_tratado = pd.read_excel(arquivo_tratado, engine='openpyxl')
        print("Arquivos carregados com sucesso.")
        return df_original, df_tratado
    except Exception as e:
        print(f"Erro ao carregar os arquivos: {e}")
        return None, None

# Função para padronizar a data
def padronizar_data(data):
    try:
        data = str(data).strip().replace(" //", "/").replace("//", "/").replace("/ ", "/").replace(" ", "")
        if len(data.split("/")) == 3:  # Se tem 3 partes (dia/mês/ano)
            dia, mes, ano = data.split("/")
            if len(ano) == 2:
                ano = "20" + ano
            return datetime.strptime(f"{dia}/{mes}/{ano}", "%d/%m/%Y").strftime("%d/%m/%Y")
        if len(data.split("/")) == 2:  # Formato incompleto, adiciona o ano atual
            data = data + "/" + str(datetime.now().year)
            return datetime.strptime(data, "%d/%m/%Y").strftime("%d/%m/%Y")
        return data
    except Exception:
        return data

# Função para padronizar os valores de quantidade
def padronizar_valores(valor):
    try:
        if pd.isna(valor) or str(valor).strip() == "":
            return "0,00"
        if isinstance(valor, str):
            valor = valor.strip().lower()
            if not valor.replace('.', '').replace(',', '').isdigit():
                return "0,00"
        valor = float(valor)
        return f"{valor:.2f}".replace('.', ',')
    except Exception:
        return "0,00"

# Função para padronizar as horas
def padronizar_horas(valor):
    try:
        if isinstance(valor, str) and valor.strip():
            if len(valor) == 2 and valor.isdigit():
                return f"{valor}:00:00"
            elif len(valor) == 5 and ":" in valor:
                return f"{valor}:00"
            elif len(valor) == 8 and ":" in valor:
                return valor
        return "00:00:00"
    except Exception:
        return "00:00:00"

# Função para verificar se os dados foram devidamente tratados
def verificar_tratamento(df_original, df_tratado, colunas_tratamento, tipo_tratamento):
    inconsistencias = []  # Lista para armazenar as inconsistências encontradas

    for coluna in colunas_tratamento:
        if coluna in df_original.columns and coluna in df_tratado.columns:
            for index, valor in enumerate(df_original[coluna]):
                valor_original = valor
                valor_tratado = df_tratado[coluna].iloc[index]
                valor_esperado = valor_tratado

                # Realizar o tratamento dos dados para comparação
                if tipo_tratamento == 'data':
                    valor_tratado = padronizar_data(valor_tratado)
                    valor_esperado = padronizar_data(valor_original)
                elif tipo_tratamento == 'valor':
                    valor_tratado = padronizar_valores(valor_tratado)
                    valor_esperado = padronizar_valores(valor_original)
                elif tipo_tratamento == 'hora':
                    valor_tratado = padronizar_horas(valor_tratado)
                    valor_esperado = padronizar_horas(valor_original)

                # Verificar falhas no tratamento
                if valor_tratado != valor_esperado:
                    inconsistencias.append({
                        "linha": index + 1,  # O índice da linha
                        "coluna": coluna,
                        "valor_original": valor_original,
                        "valor_tratado": valor_tratado,
                        "valor_esperado": valor_esperado
                    })

    return inconsistencias

# Função para salvar as inconsistências em um arquivo de texto
def salvar_falhas_em_arquivo(inconsistencias, arquivo_saida):
    try:
        with open(arquivo_saida, 'w') as f:
            if inconsistencias:
                for inconsistencia in inconsistencias:
                    f.write(f"Falha encontrada:\n")
                    f.write(f"Coluna: {inconsistencia['coluna']}\n")
                    f.write(f"Linha: {inconsistencia['linha']}\n")
                    f.write(f"Valor original: {inconsistencia['valor_original']}\n")
                    f.write(f"Valor tratado: {inconsistencia['valor_tratado']}\n")
                    f.write(f"Valor esperado: {inconsistencia['valor_esperado']}\n")
                    f.write("-" * 50 + "\n")
            else:
                f.write("Nenhuma inconsistência encontrada.\n")
        print(f"Falhas registradas no arquivo: {arquivo_saida}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo de falhas: {e}")

# Caminhos dos arquivos
diretorio_atual = Path.cwd()
arquivo_original = diretorio_atual / "relatorio_final.csv"
arquivo_tratado = diretorio_atual / "relatorio_tratado.xlsx"
arquivo_saida = diretorio_atual / "falhas_encontradas.txt"

# Carregar os dados dos arquivos
df_original, df_tratado = carregar_dados(arquivo_original, arquivo_tratado)

# Verificar se os arquivos foram carregados corretamente
if df_original is not None and df_tratado is not None:
    # Definir as colunas que precisam ser verificadas
    colunas_tratamento_data = ["DATA DE EXECUÇÃO"]
    colunas_tratamento_valores = ["QTD PESSOAS", "QUANTIDADE DE ANDAIMES"]
    colunas_tratamento_horas = ["HORÁRIO DE DISPONIBILIDADE", "HORÁRIO DE INÍCIO DA ATIVIDADE"]

    # Verificar os dados conforme o tipo de tratamento
    inconsistencias_data = verificar_tratamento(df_original, df_tratado, colunas_tratamento_data, 'data')
    inconsistencias_valores = verificar_tratamento(df_original, df_tratado, colunas_tratamento_valores, 'valor')
    inconsistencias_horas = verificar_tratamento(df_original, df_tratado, colunas_tratamento_horas, 'hora')

    # Combinar todas as inconsistências
    inconsistencias = inconsistencias_data + inconsistencias_valores + inconsistencias_horas

    # Salvar as falhas em um arquivo de texto
    salvar_falhas_em_arquivo(inconsistencias, arquivo_saida)


Arquivos carregados com sucesso.
Falhas registradas no arquivo: c:\Users\Administrador\Desktop\filtro espiral em jupyter\falhas_encontradas.txt


Correção de inconsistências

In [213]:
import pandas as pd

# Função para corrigir o valor no formato adequado (duas casas decimais com vírgula)
def corrigir_valor(valor):
    try:
        # Verifica se o valor pode ser convertido em float e aplica o formato desejado
        valor_corrigido = f"{float(valor):.2f}".replace('.', ',')
        return valor_corrigido
    except ValueError:
        # Caso não consiga converter para float, retorna o valor original
        return valor

# Carregar o arquivo de falhas encontradas
with open('/content/falhas_encontradas.txt', 'r') as file:
    falhas = file.readlines()

# Carregar o arquivo de dados
df = pd.read_excel('/content/relatorio_tratado.xlsx')

# Processar as falhas encontradas e corrigir os valores no DataFrame
for i in range(len(falhas)):
    linha = falhas[i].strip()

    if 'Falha encontrada' in linha:
        # Extrair as informações da falha
        coluna = falhas[i + 1].split(":")[1].strip()
        linha_falha = int(falhas[i + 2].split(":")[1].strip()) - 1  # Ajusta para índice 0 do pandas
        valor_original = falhas[i + 3].split(":")[1].strip()
        valor_tratado = falhas[i + 4].split(":")[1].strip()
        valor_esperado = falhas[i + 5].split(":")[1].strip()

        # Corrigir o valor no DataFrame com o valor esperado
        if coluna in df.columns:  # Verificar se a coluna existe no DataFrame
            df.at[linha_falha, coluna] = corrigir_valor(valor_esperado)

# Salvar o arquivo corrigido
df_corrigido = df.copy()  # Faz uma cópia do DataFrame já corrigido
df_corrigido.to_excel('/content/relatorio_tratado_corrigido.xlsx', index=False)

print("Arquivo tratado corrigido salvo em: /content/relatorio_tratado_corrigido.xlsx")


FileNotFoundError: [Errno 2] No such file or directory: '/content/falhas_encontradas.txt'